In [ ]:
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from slsim.Lenses.lens_pop import LensPop
import numpy as np
import slsim.Sources as sources
import slsim.Deflectors as deflectors
import slsim.Pipelines as pipelines
from slsim.Sources.SourceCatalogues.QuasarCatalog.quasar_pop import QuasarRate
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

## Lens population --> Catalog


Here we demonstrate functionality of requesting a catalog from a population of lenses generated by SLSim.
We store properties of each lens object, and then accumulate the properties of each ``Lens`` in the population into a ``pandas.DataFrame``.
We store the DataFrame as a pickle file -- this means that we can store the Lens object itself in the DataFrame, reload the file, and still have all access to the ``Lens`` methods and functionalities.
Thus, we can store a ``LensPop`` sample, look at corner plots to understand the distribution of parameters in the ``LensPop``, and finally load in the ``Lens`` systems to further work on light curve simulation, image simulation etc.

If you have any questions or concerns feel free to reach out to Padma [pv10@illinois.edu].

### Loading in sky area

In [ ]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)


# define a sky area
galaxy_sky_area = Quantity(value=1, unit="deg2")
quasar_sky_area = Quantity(value=5, unit="deg2")
full_sky_area = Quantity(value=500, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition
# to the skypy config
# file)

kwargs_deflector_cut = {"band": "i", "band_max": 30, "z_min": 0.01, "z_max": 5}
kwargs_source_cut = {"z_min": 0.001, "z_max": 5.0}

### Instantiate ``SkyPyPipeline`` to create sample of potential deflectors ``deflectors.AllLensGalaxies``

In [ ]:
# generate galaxy population using skypy pipeline.
galaxy_simulation_pipeline = pipelines.SkyPyPipeline(
    skypy_config=None,
    sky_area=galaxy_sky_area,
    filters=None,
)

In [ ]:
# Initiate deflector popiulation class
lens_galaxies = deflectors.AllLensGalaxies(
    red_galaxy_list=galaxy_simulation_pipeline.red_galaxies,
    blue_galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light={},
    cosmo=cosmo,
    sky_area=galaxy_sky_area,
)

### ``QuasarRate`` will generate N number of point sources over some sky area and redshift range.
``quasar_sample`` can be used to filter this sample of point sources according to observational magnitude cuts.

In [ ]:
# Initiate QuasarRate class to generate quasar sample.
quasar_class = QuasarRate(
    cosmo=cosmo,
    sky_area=quasar_sky_area,
    noise=True,
    redshifts=np.linspace(0.001, 5.01, 100),  # these redshifts are provided
    # to match general slsim redshift range in skypy pipeline.
)
quasar_source = quasar_class.quasar_sample(m_min=15, m_max=30)

Once you've initialized a quasar sample with properties M_i and z, you can go on to assign some quasar variability properties, and generate a ``sources.PointSources`` class. This is the parallel to ``deflectors.AllLensGalaxies``.

In [ ]:
# Prepare dictionary of agn variability kwargs
variable_agn_kwarg_dict = {
    "length_of_light_curve": 1000,
    "time_resolution": 1,
    "log_breakpoint_frequency": 1 / 20,
    "low_frequency_slope": 1,
    "high_frequency_slope": 3,
    "standard_deviation": 0.9,
}

kwargs_quasar = {
    "variability_model": "light_curve",
    "kwargs_variability": {"agn_lightcurve", "i", "r"},
    "agn_driving_variability_model": "bending_power_law",
    "agn_driving_kwargs_variability": variable_agn_kwarg_dict,
    "lightcurve_time": np.linspace(0, 1000, 5),
}
# Initiate source population class.
source_quasar = sources.PointSources(
    quasar_source,
    cosmo=cosmo,
    sky_area=quasar_sky_area,
    kwargs_cut=kwargs_source_cut,
    point_source_type="quasar",
    point_source_kwargs=kwargs_quasar,
)

### Generate lens population, with some optional cuts

In [ ]:
# Initiate LensPop class to generate lensed quasar pop. We simulate lens pop in 500
# deg^2. If you want to simulate in larger sky, change sky area to your requirement.
quasar_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_quasar,
    cosmo=cosmo,
    sky_area=full_sky_area,
)
kwargs_lens_cuts = {}

# drawing population
quasar_lens_population = quasar_lens_pop.draw_population(
    speed_factor=1000, kwargs_lens_cuts=kwargs_lens_cuts
)

In [ ]:
### view the first lensed system Lens object
chosen_lens = quasar_lens_population[0]

#### Lens properties in a dataframe for a single lens:

In [ ]:
### Now we can see all the properties of a lens by calling lens.lens_to_dataframe()
### If you want to view the optional inputs or the function description of method lens_to_dataframe,
### run the following: chosen_lens.lens_to_dataframe?
chosen_lens.lens_to_dataframe()

### Population catalog generation

In [ ]:
# to get the catalog for a whole population, we can loop this
# this takes around a minute to run for ~ 3000 lenses
full_pop_df = pd.DataFrame()
for i, lens_obj in enumerate(quasar_lens_population):
    full_pop_df = lens_obj.lens_to_dataframe(index=i, df=full_pop_df)

In [ ]:
### if you want to just look at the microlensing properties
### in general, you can call full_pop_df.columns to look at all available column names
micro_cols = [c for c in full_pop_df.columns if "micro" in c]
full_pop_df[micro_cols].head(5)

In [ ]:
### if you want to look at quads
quads_df = full_pop_df[full_pop_df["num_ps_images"] == 4]  # total 182 rows
quads_df.head(5)

#### Save the population dataframe to a pickle file. This allows us to also save the Lens object and retain its functionality

In [ ]:
### Here, we explore additional functionality of saving the object to the df, and pickling the file
full_pop_df["lens_obj"] = None
for i, lens_obj in enumerate(quasar_lens_population):
    full_pop_df.loc[i, "lens_obj"] = lens_obj

In [ ]:
full_pop_df.head()  # there are 3509 rows in total in this run
# recall that no observing cuts have been placed when querying lenses

In [ ]:
### save to pickle file -- ideally this would not be in the same notebooks folder, but some other path
path_to_stored_catalog = "lens_pop_data.pkl"
full_pop_df.to_pickle(path_to_stored_catalog)

## Restart kernel and see if loaded in data is fully functional

In [ ]:
### uncomment and load this cell!

# from astropy.cosmology import FlatLambdaCDM
# from astropy.units import Quantity
# import slsim
# from slsim.Lenses.lens_pop import LensPop
# import numpy as np
# import slsim.Sources as sources
# import slsim.Deflectors as deflectors
# import slsim.Pipelines as pipelines
# from slsim.Sources.SourceCatalogues.QuasarCatalog.quasar_pop import QuasarRate
# import corner
# import matplotlib.pyplot as plt
# from scipy.stats import gaussian_kde
# import pandas as pd

# %load_ext autoreload
# %autoreload 2

In [ ]:
### load in pickle file
path_to_stored_catalog = "lens_pop_data.pkl"
read_in_data = pd.read_pickle(path_to_stored_catalog)
read_in_data.head()

In [ ]:
# make sure you've imported all the SLSim methods 2 cells above
# load in lens object and make sure method works.
# here, we can see that the loaded in lens object is still able to produce magnitude at different times.
first_lens = read_in_data.loc[0, "lens_obj"]
first_lens.point_source_magnitude(
    band="i", lensed=True, time=np.linspace(0, 1000, 1000)
)

We have demonstrated that lens objects retrieved from the dataframe still have full functionality in a new environment. This enables users to store and load Lenses for future analysis.